In [55]:
import numpy as np

## lambda pour depart
Lam = np.array([2.8, 3.7, 5.5, 3.5, 4.6], dtype=float)

## p_ij
Prob = np.array([
    [0, 0.2, 0.3, 0.2, 0.3], 
    [0.2, 0, 0.3, 0.2, 0.3],
    [0.2, 0.25, 0, 0.25, 0.3],
    [0.15, 0.2, 0.3, 0, 0.35],
    [0.2, 0.25, 0.35, 0.2, 0]
], dtype=float)

## mu_ij -> par heure
Mu = np.array([
    [0, 3, 5, 7, 7],
    [2, 0, 2, 5, 5],
    [4, 2, 0, 3, 3],
    [8, 6, 4, 0, 2],
    [7, 7, 5, 2, 0]
], dtype=float)
Mu = np.divide(Mu, 60.0)
Mu = np.divide(1, Mu, where=Mu!=0)

## l'etat initial
State = np.array([
    [20, 1, 0, 0, 0],
    [1, 15, 1, 0, 0],
    [0, 1, 17, 1, 0],
    [0, 0, 1, 13, 1],
    [0, 0, 0, 1, 18]
])

## taux de transition
def transRate(state, lam=Lam, mu=Mu, p=Prob):
    lamExt = np.transpose(np.ones((5, 5)) * Lam)
    notEmpty = [np.sign(state[i][i]) for i in range(5)]
    depart = np.multiply(np.multiply(lamExt, notEmpty), p)
    arrive = np.multiply(state, mu)
    res = np.concatenate((depart, arrive), axis=1)
    return np.sum(res), res

## duree dans un etat
def last(lam):
    return np.random.exponential(1/lam)

## l'etat suivant
def nextState(rate, state):
    choice = np.random.choice(np.arange(50), p=rate.flatten()/np.sum(rate))
    i, j = choice / 10, choice % 10
    action = np.zeros((5, 5))
    if j <= 4:
        action[i][i], action[i][j] = -1, 1
    else:
        action[i][j-5], action[j-5][j-5] = -1, 1
    
#     print("rate")
#     print(rate)
#     print("action (%d %d, %d)" % (choice, i, j))
    
    return np.add(state, action)
    
## simulation
def simulate(init=State, lam=Lam, mu=Mu, p=Prob, t=1.0):
    time = 0.0
    states = [[time, init]]
    while time <= t:
#         print("time %f" % time)
#         print(states[-1][1])
        stayLam, changeRate = transRate(state=states[-1][1], lam=Lam, mu=Mu, p=Prob)
        lastTime = last(lam=stayLam)
        time = time + lastTime
        
        state = nextState(changeRate, states[-1][1])
        states.append([time, state])
        
#         print("state")
#         print(state)
#         print("***************************")
    
    states[-1][0] = t
    
    return states    

In [59]:
## commencer a simuler pendant une heure
T = 1.0
print("Simulation during %.2f hour(s)." % T)
states = simulate(t=T)
for state in states:    
    print("time = %f" % state[0])
    print("state")
    print(state[1])
    print("-------------------------")

Simulation during 1.00 hour(s).
time = 0.000000
state
[[20  1  0  0  0]
 [ 1 15  1  0  0]
 [ 0  1 17  1  0]
 [ 0  0  1 13  1]
 [ 0  0  0  1 18]]
-------------------------
time = 0.003970
state
[[20.  1.  0.  0.  0.]
 [ 1. 15.  0.  0.  0.]
 [ 0.  1. 18.  1.  0.]
 [ 0.  0.  1. 13.  1.]
 [ 0.  0.  0.  1. 18.]]
-------------------------
time = 0.008126
state
[[20.  1.  0.  0.  0.]
 [ 1. 16.  0.  0.  0.]
 [ 0.  0. 18.  1.  0.]
 [ 0.  0.  1. 13.  1.]
 [ 0.  0.  0.  1. 18.]]
-------------------------
time = 0.013547
state
[[20.  0.  0.  0.  0.]
 [ 1. 17.  0.  0.  0.]
 [ 0.  0. 18.  1.  0.]
 [ 0.  0.  1. 13.  1.]
 [ 0.  0.  0.  1. 18.]]
-------------------------
time = 0.026545
state
[[20.  0.  0.  0.  0.]
 [ 1. 17.  0.  0.  0.]
 [ 0.  0. 19.  1.  0.]
 [ 0.  0.  0. 13.  1.]
 [ 0.  0.  0.  1. 18.]]
-------------------------
time = 0.027891
state
[[20.  0.  0.  0.  0.]
 [ 1. 17.  0.  0.  0.]
 [ 0.  0. 19.  1.  0.]
 [ 0.  0.  0. 13.  1.]
 [ 0.  0.  1.  1. 17.]]
-------------------------
time = 0.